In [1]:
import os

import cdsapi
import xarray as xr

_ = xr.set_options(keep_attrs=True)
os.chdir(os.getenv("DATA_VOLUME", "../../../.."))
!pwd

/Users/nicola/devel/esa-hedp-zarr3/src/earthdatahub-catalogue


In [2]:
!uv pip list| grep aws

Using Python 3.13.7 environment at: /Users/nicola/opt/miniconda3/envs/new-zarr-3-env
awscli                    2.31.4
awscli-plugin-endpoint    0.4
awscrt                    0.27.6


In [3]:
#!aws s3 ls

In [6]:
storage_options = {
    "key": "3c3d8fe87ec04e9b9c3a07e10ecc8daa",
    "secret": "f8d4d5713147465aafc9e7f3649223e7"
    }

In [7]:
ecv_store = "s3://edh-root/more-era5/ecv-for-climate-change-1981-2010-anomaly-v0/"
ds = xr.open_dataset(
    ecv_store,
    engine="zarr",
    storage_options = storage_options
)
ds

<xarray.Dataset> Size: 9GB
Dimensions:     (valid_time: 562, latitude: 721, longitude: 1440)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 4kB 1979-01-01 ... 2025-10-01
  * latitude    (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude   (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
Data variables:
    r           (valid_time, latitude, longitude) float32 2GB ...
    siconc      (valid_time, latitude, longitude) float32 2GB ...
    t2m         (valid_time, latitude, longitude) float32 2GB ...
    tp          (valid_time, latitude, longitude) float32 2GB ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-11-06T12:03 GRIB to CDM+CF via cfgrib-0.9.1...

In [8]:
dataset = "ecv-for-climate-change"
base_request = {
        'format': 'zip',
        # 'variable': 'surface_air_temperature',
        'product_type': 'anomaly',
        'climate_reference_period': '1981_2010',
        'time_aggregation': '1_month_mean',
        # 'year': ['2023'],
        # 'month': ['01'],
        'origin': 'era5',
    }

In [10]:
year_month = "2025-11"

In [11]:
try:
    ds.sel(valid_time=year_month)
except KeyError:
    print("OK, data not present")
else:
    raise ValueError("STOP, the data is already present!")

print("LAST DATA IS", str(ds.valid_time.values[-1])[:7], "and you are adding", year_month)

OK, data not present
LAST DATA IS 2025-10 and you are adding 2025-11


In [12]:
year, _, month = year_month.partition("-")
year_month_request = base_request | {"year": [year], "month": [month]}
year_month_request

{'format': 'zip',
 'product_type': 'anomaly',
 'climate_reference_period': '1981_2010',
 'time_aggregation': '1_month_mean',
 'origin': 'era5',
 'year': ['2025'],
 'month': ['11']}

In [13]:
ecv_0 = [
    "surface_air_temperature",
    "surface_air_relative_humidity",
    "sea_ice_cover"
]
target_0 = f"ecv-0-{year_month}"
target_0

'ecv-0-2025-11'

In [14]:
client = cdsapi.Client()
client

2025-12-09 10:08:47,988 INFO [2025-12-03T00:00:00Z] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.


In [15]:
dataset

'ecv-for-climate-change'

In [16]:
year_month_request | {"variable": ecv_0}

{'format': 'zip',
 'product_type': 'anomaly',
 'climate_reference_period': '1981_2010',
 'time_aggregation': '1_month_mean',
 'origin': 'era5',
 'year': ['2025'],
 'month': ['11'],
 'variable': ['surface_air_temperature',
  'surface_air_relative_humidity',
  'sea_ice_cover']}

In [17]:
target_0 + ".zip"

'ecv-0-2025-11.zip'

In [18]:
result = client.retrieve(dataset, year_month_request | {"variable": ecv_0}, target=target_0 + ".zip")
!unzip -o {target_0}.zip -d {target_0}

2025-12-09 10:08:57,238 INFO Request ID is 82c770d4-c83a-4e8a-b032-543712bdda5f
2025-12-09 10:08:57,298 INFO status has been updated to accepted
2025-12-09 10:09:18,493 INFO status has been updated to running
2025-12-09 10:09:29,942 INFO status has been updated to successful


883a5f0ee5b40b6723f51d206f939bc5.zip:   0%|          | 0.00/5.61M [00:00<?, ?B/s]

Archive:  ecv-0-2025-11.zip
  inflating: ecv-0-2025-11/1month_anomaly_Global_ea_2t_202511_1981-2010_v02.grib  
  inflating: ecv-0-2025-11/1month_anomaly_Global_ea_ci_202511_1981-2010_v02.grib  
  inflating: ecv-0-2025-11/1month_anomaly_Global_ea_r2_202511_1981-2010_v02.grib  


In [19]:
ds_ecv_0 = xr.open_mfdataset(f"{target_0}/*grib", time_dims=["valid_time"], squeeze=False).isel(number=0, surface=0)
ds_ecv_0

<xarray.Dataset> Size: 12MB
Dimensions:     (valid_time: 1, latitude: 721, longitude: 1440)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 8B 2025-11-01
  * latitude    (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude   (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    number      int64 8B 0
    surface     float64 8B 0.0
Data variables:
    t2m         (valid_time, latitude, longitude) float32 4MB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    siconc      (valid_time, latitude, longitude) float32 4MB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    r           (valid_time, latitude, longitude) float32 4MB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-12-09T10:16 GRIB to CDM+CF via cfgrib-0.9.1...

In [20]:
ecv_6 = ["precipitation"]
target_6 = f"ecv-6-{year_month}"
target_6

'ecv-6-2025-11'

In [21]:
result = client.retrieve(dataset, year_month_request | {"variable": ecv_6}, target=target_6 + ".zip")
!unzip {target_6}.zip -d {target_6}

2025-12-09 10:16:28,398 INFO Request ID is 3685bc3b-ce87-40e3-b8c1-241be762779d
2025-12-09 10:16:28,452 INFO status has been updated to accepted
2025-12-09 10:16:36,833 INFO status has been updated to running
2025-12-09 10:17:01,020 INFO status has been updated to successful


6ea4fea4d93c0e0d4431f9909383d619.zip:   0%|          | 0.00/1.52M [00:00<?, ?B/s]

Archive:  ecv-6-2025-11.zip
  inflating: ecv-6-2025-11/1month_anomaly_Global_ea_tp_202511_1981-2010_v02.grib  


In [22]:
ds_ecv_6 = xr.open_mfdataset(f"{target_6}/*grib", time_dims=["valid_time"], squeeze=False).isel(number=0, surface=0)
ds_ecv_6 = ds_ecv_6.assign_coords(valid_time=ds_ecv_0.valid_time)
ds_ecv_6

<xarray.Dataset> Size: 4MB
Dimensions:     (latitude: 721, longitude: 1440, valid_time: 1)
Coordinates:
  * latitude    (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude   (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * valid_time  (valid_time) datetime64[ns] 8B 2025-11-01
    number      int64 8B 0
    surface     float64 8B 0.0
Data variables:
    tp          (valid_time, latitude, longitude) float32 4MB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-12-09T10:19 GRIB to CDM+CF via cfgrib-0.9.1...

In [23]:
ds_ecv = xr.merge([ds_ecv_0, ds_ecv_6]).drop_vars(["number", "surface"])
ds_ecv

<xarray.Dataset> Size: 17MB
Dimensions:     (valid_time: 1, latitude: 721, longitude: 1440)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 8B 2025-11-01
  * latitude    (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude   (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
Data variables:
    t2m         (valid_time, latitude, longitude) float32 4MB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    siconc      (valid_time, latitude, longitude) float32 4MB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    r           (valid_time, latitude, longitude) float32 4MB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    tp          (valid_time, latitude, longitude) float32 4MB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-12-09T10:16 GRIB to CDM+CF via cfgrib-0.9.1...

In [27]:
#
# uncommento to UPDATE PRODUCTION!
#
'''
ds_ecv.to_zarr(
    ecv_store, mode="a", 
    append_dim="valid_time",
    storage_options = storage_options
)
'''

'\nds_ecv.to_zarr(\n    ecv_store, mode="a", \n    append_dim="valid_time",\n    storage_options = storage_options\n)\n'

In [25]:
xr.open_dataset(
    ecv_store,
    engine="zarr",
    storage_options = storage_options
)

<xarray.Dataset> Size: 9GB
Dimensions:     (valid_time: 563, latitude: 721, longitude: 1440)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 5kB 1979-01-01 ... 2025-11-01
  * latitude    (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude   (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
Data variables:
    r           (valid_time, latitude, longitude) float32 2GB ...
    siconc      (valid_time, latitude, longitude) float32 2GB ...
    t2m         (valid_time, latitude, longitude) float32 2GB ...
    tp          (valid_time, latitude, longitude) float32 2GB ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-12-09T10:16 GRIB to CDM+CF via cfgrib-0.9.1...